# Linear Regression i Logistic Regression en PyTorch
### Sessió 2 Problemes

Hem extret aquesta petita intro d'aquest blog:

https://medium.com/biaslyai/pytorch-linear-and-logistic-regression-models-5c5f0da2cb9

## Outline
* Linear Regression
* Define Model Structure
* Loss Function (Criterion) and Optimizer
* Model Training
* Make Predictions
* Logistic Regression


## Linear Regression

<img src="https://miro.medium.com/max/640/1*QiU6DcP_r9qWLznMw0-M_Q.png">

Linear regression is a method commonly used for predictive statistical analysis. The objective of the regression task is to explain and make adequate predictions based on the linear relation with an independent variable. In other words, we would like to find the best-fitting line through the independent variables present in our data. The simplest form of the linear regression equation is defined by:

$Ŷ = bX + a + e$

where,

$Ŷ$ = Predicted value of $Y$

$X$ = Independent variable

$b$ = Slope coefficient based on best-fitting line

$a$ = Intercept

$e$ = Error term


## Define Model Structure
Now, let’s observe how we set up the base structure of this model in PyTorch. We first import library functions and define some toy data points. Here, x_data is the independent variable, and y_data is the target variable to be learned by the model. The model learns the best-fitting regression line through the slope coefficient and Y-intercept. Then, it can make new predictions with newly introduced x_data points.


In [1]:
import torch

x_data = torch.Tensor([[0.1], [0.4], [0.6], [0.7], [0.3], [.15], [0.5], [.45]])
y_data = torch.Tensor([[2.6], [3.5], [4.0], [4.5], [3.1], [2.8], [4.1], [3.7]])

In order to set up the model class, we need to initialize the model type and declare the forward pass. Since our model takes one independent variable and makes one prediction for the Ŷ variable at a time, we initialize our model with this linear layer: torch.nn.Linear(1, 1) , where the first 1 is for the input size, and the second 1 is for the output size.
Next is to define the forward pass function. The forward pass refers to the calculation process of the output data from the input. We simply define as below. The function takes x as its input and outputs the predicted value of Y, y_pred .


In [2]:
class LinearRegression(torch.nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = torch.nn.Linear(1, 1)
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred
    
model = LinearRegression()

## Loss Function (Criterion) and Optimizer
After the forward pass, a loss function is calculated from the target y_data and the prediction y_pred in order to update weights for the best model selection in the further step. Setting up the loss function is a fairly simple step in PyTorch. Here, we will use the Mean Square Error (MSE), which is the most commonly used regression loss function.

In [3]:
criterion = torch.nn.MSELoss(size_average=False)

/anaconda3/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Next, we will use Stochastic Gradient Descent (SGD) optimizer for the update of hyperparameters. model.parameters() will provide the learnable parameters to the optimizer and lr=0.01 defines the learning rates for the parameter updates.

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

## Model Training
Our model is now ready to train. We begin by setting up an epoch size. Epoch is a single pass through whole training dataset. In the example below, the epoch size is set to 20, meaning there will be 20 single passes of the training and weight updates.

In [5]:
model.train()  # ens posem en mode 'train'
for epoch in range(20):  # fem 20 epoques 
    optimizer.zero_grad()  # resetejem els gradients a zero
    # Forward pass
    y_pred = model(x_data) # fem la predicció
    # Compute Loss
    loss = criterion(y_pred, y_data) # mirem el error
    # Backward pass
    loss.backward()     # calculem els gradients
    optimizer.step()    # actualitzem els pesos

After the forward pass and the loss computation is done, we do a backward pass, which refers to the process of learning and updating the weights.

We first need to set our gradient to zero: optimizer.zero_grad(). This is because every time a variable is back propagated through, the gradient will be accumulated instead of being replaced. Then we run a backward pass by loss.backward() .

The optimizer.step() performs a parameter update based on the current gradient.

## Make Predictions
Now that our model is trained, we can simply make a new prediction by inputting a new x value to our model:

In [6]:
new_x = torch.Tensor([[1.0]])
y_pred = model(new_x)
print("predicted Y value: ", y_pred.data[0][0])

predicted Y value:  tensor(4.2771)


# Logistic Regression
Now that we know how to build a linear regression model, let’s look into building a logistic regression model. Just like a linear regression model, logistic regression is predictive analysis. It is used when the dependent variable Y is dichotomous (binary).

<img src="https://miro.medium.com/max/400/1*GHVJ6jGVsxbuoJj5d3cvzg.png">

In [7]:
x_data_cls = torch.Tensor([[-3], [2], [0], [4], [-2], [1], [5], [1.5]])
y_data_cls = torch.Tensor([[ 0], [1], [0], [1],  [0], [0], [1], [1]])

This can be done by using a sigmoid function which outputs values between 0 and 1. Any output >0.5 will be class 1 and class 0 otherwise. Thus, the logistic regression equation is defined by:

$Ŷ = σ ( bX + a + e)$

$ σ = \frac{1}{1+e^{-z}} $ 

In the code, a simple modification to the linear regression model creates a logistic regression model. We can simply apply functional.sigmoid to our current linear output from the forward pass: y_pred = functional.sigmoid(self.linear(x)) . The complete model class is defined below:

In [8]:
class LogisticRegression(torch.nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(1, 1)
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model_cls = LogisticRegression()


For the loss function, we use Binary Cross-Entropy (BCE), which is known as the binary logarithmic loss function. This is commonly used for logistic regression tasks since we are predicting a binary value as output.

In [9]:
criterion_cls = torch.nn.BCELoss(size_average=True)

/anaconda3/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [10]:
optimizer = torch.optim.SGD(model_cls.parameters(), lr=0.01)

The rest of the code will be the same as the linear model.


In [11]:
model_cls.train()  # ens posem en mode 'train'
for epoch in range(200):
    optimizer.zero_grad()
    # Forward pass
    y_pred_cls = model_cls(x_data_cls)
    # Compute Loss
    loss = criterion_cls(y_pred_cls, y_data_cls)
    # C = 1  # Regularization factor
    #loss += torch.sum(model_cls.linear.weight**2) * (1 / C)
    
    # Backward pass
    loss.backward()
    optimizer.step()

new_x_cls = torch.Tensor([[1.0]])
y_pred_cls = torch.sigmoid(model_cls(x_data_cls))
print("predicted Y value: ", y_pred_cls.data[0][0])
print("predicted Y value Binary: ", y_pred_cls.data[0][0] > 0.5)
print(f"weigths learned: \nb(slope)={model_cls.linear.weight} \na(intercept)={model_cls.linear.bias}")

predicted Y value:  tensor(0.5099)
predicted Y value Binary:  tensor(True)
weigths learned: 
b(slope)=Parameter containing:
tensor([[0.8845]], requires_grad=True) 
a(intercept)=Parameter containing:
tensor([-0.5381], requires_grad=True)
